Baseline - ARIMA

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import acf
from statsmodels.graphics.tsaplots import plot_acf
from scipy.stats import linregress
import os

# Optional for larger text and better visuals
sns.set(style="whitegrid", context="talk")

In [2]:
# Load the sales data (tab-delimited)
sales = pd.read_csv("datasets/sell-in.txt", sep="\t", dtype={"periodo": str})

# Load the stocks data (tab-delimited)
stocks = pd.read_csv("datasets/tb_stocks.txt", sep="\t", dtype={"periodo": str})

# Load the product information data (tab-delimited)
product_info = pd.read_csv("datasets/tb_productos.txt", sep="\t")

#carga productos a predecir
product_predict = pd.read_csv("datasets/product_id_apredecir201912.txt", sep="\t",header=0)

In [3]:
# Asegurar que 'periodo' sea string
sales['periodo'] = sales['periodo'].astype(str)



In [4]:
# Paso 1: agrupar por producto y periodo, sumando tn por mes
tn_por_producto_y_periodo = (
    sales
    .groupby(['product_id', 'periodo'])['tn']
    .sum()
    .reset_index()
)


In [5]:
tn_por_producto_y_periodo.head()

,product_id,periodo,tn
0,20001,201701,934.77222
1,20001,201702,798.01620
2,20001,201703,1303.35771
3,20001,201704,1069.96130
4,20001,201705,1502.20132


In [6]:
# Filtrar tn_por_producto_y_periodo
tn_por_producto_y_periodo_a_predecir= tn_por_producto_y_periodo[tn_por_producto_y_periodo['product_id'].isin(product_predict['product_id'])]

# Mostrar el resultado
print(tn_por_producto_y_periodo_a_predecir)

       product_id periodo          tn
0           20001  201701   934.77222
1           20001  201702   798.01620
2           20001  201703  1303.35771
3           20001  201704  1069.96130
4           20001  201705  1502.20132
...           ...     ...         ...
31206       21276  201908     0.01265
31207       21276  201909     0.01856
31208       21276  201910     0.02079
31209       21276  201911     0.03341
31210       21276  201912     0.00892

[22349 rows x 3 columns]


In [7]:
#tn_por_producto_y_periodo_a_predecir.to_csv("datasets/tn_por_producto_y_periodo_a_predecir.csv", index=False, encoding="utf-8")

In [8]:
sales.head()

,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn
0,201701,10234,20524,0,2,0.05300,0.05300
1,201701,10032,20524,0,1,0.13628,0.13628
2,201701,10217,20524,0,1,0.03028,0.03028
3,201701,10125,20524,0,1,0.02271,0.02271
4,201701,10012,20524,0,11,1.54452,1.54452


In [9]:
from statsmodels.tsa.arima.model import ARIMA
from tqdm import tqdm

# Limpiar columnas
sales.columns = sales.columns.str.strip()
product_predict.columns = product_predict.columns.str.strip()

In [10]:
# Convertir periodo a string y luego a datetime con formato año-mes
sales['periodo'] = sales['periodo'].astype(str)
sales['periodo'] = pd.to_datetime(sales['periodo'], format='%Y%m')

In [11]:
# Filtrar productos a predecir
sales = sales[sales['product_id'].isin(product_predict['product_id'])]

In [12]:
# Agrupar por producto y periodo
ventas_agrupadas = (
    sales
    .groupby(['product_id', 'periodo'])['tn']
    .sum()
    .reset_index()
    .sort_values(['product_id', 'periodo'])
)

In [13]:
# Lista para guardar predicciones
resultados = []

# Predicción ARIMA por producto
for producto in tqdm(product_predict['product_id']):
    serie = ventas_agrupadas[ventas_agrupadas['product_id'] == producto]
    serie = serie.set_index('periodo')
    ts = serie['tn']

    # Validaciones mínimas
    if ts.isnull().any() or len(ts) < 6 or ts.sum() == 0 or ts.nunique() == 1:
        forecast = 0
    else:
        try:
            model = ARIMA(ts, order=(1, 1, 1))
            model_fit = model.fit()
            forecast = model_fit.forecast(steps=1)[0]
            forecast = max(forecast, 0)
        except:
            forecast = 0

    resultados.append({'product_id': producto, 'tn': forecast})



  0%|          | 0/780 [00:00<?, ?it/s]c:\Users\paola\anaconda3\envs\ldi2\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\paola\anaconda3\envs\ldi2\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\paola\anaconda3\envs\ldi2\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
C:\Users\paola\AppData\Local\Temp\ipykernel_16124\2606602479.py:17: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  forecast = 

In [ ]:
# Guardar resultados
df_predicciones = pd.DataFrame(resultados)
df_predicciones.to_csv("data/pred_arima111.csv", index=False, encoding="utf-8")
print("✅ Archivo submission_arima.csv generado correctamente.")

✅ Archivo submission_arima.csv generado correctamente.
